In [11]:
#!pip install transformers
#pip install -U sacremoses

In [6]:
import pandas as pd

csv_path = "/home/alex/dev/AiLearning/DataSetStore/twitter_msg_sentiment_anal/train.csv"

df = pd.read_csv(csv_path)
df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [7]:

# https://huggingface.co/docs/transformers/model_doc/funnel

from transformers import AutoTokenizer, FunnelBaseModel
import torch

tokenizer = AutoTokenizer.from_pretrained("funnel-transformer/small-base")
model = FunnelBaseModel.from_pretrained("funnel-transformer/small-base")
inputs = tokenizer("Hello, my dog is cute cccccc", return_tensors="pt")
#outputs = model(**inputs)

#last_hidden_states = outputs.logits
print (inputs)
print(model)

{'input_ids': tensor([[  101,  7592,  1010,  2026,  3899,  2003, 10140, 10507,  9468,  9468,
           102]]), 'token_type_ids': tensor([[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
FunnelBaseModel(
  (embeddings): FunnelEmbeddings(
    (word_embeddings): Embedding(30522, 768)
    (layer_norm): LayerNorm((768,), eps=1e-09, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): FunnelEncoder(
    (attention_structure): FunnelAttentionStructure(
      (sin_dropout): Dropout(p=0.1, inplace=False)
      (cos_dropout): Dropout(p=0.1, inplace=False)
    )
    (blocks): ModuleList(
      (0-2): 3 x ModuleList(
        (0-3): 4 x FunnelLayer(
          (attention): FunnelRelMultiheadAttention(
            (hidden_dropout): Dropout(p=0.1, inplace=False)
            (attention_dropout): Dropout(p=0.1, inplace=False)
            (q_head): Linear(in_features=768, out_features=768, bias=False)
            (k_hea

In [12]:
# from transformers import AutoTokenizer, FSMTModel

# import torch

# tokenizer = AutoTokenizer.from_pretrained("facebook/wmt19-ru-en")

# model = FSMTModel.from_pretrained("facebook/wmt19-ru-en")
# inputs = tokenizer("Hello, ", return_tensors="pt")
# outputs = model(**inputs)

# last_hidden_states = outputs.last_hidden_state
# print (last_hidden_states.shape)
# print(model)

In [70]:
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("funnel-transformer/small-base")
model = FunnelBaseModel.from_pretrained("funnel-transformer/small-base")

def funnel_encoder (txt: str, verbose=True, new_size=None):

    with torch.no_grad():
        inputs = tokenizer(txt, return_tensors="pt")
        outputs = model(**inputs)
        result = outputs.last_hidden_state.numpy().copy()
        result = result.reshape(result.shape[0]*result.shape[1]*result.shape[2])
        zero = result
        if new_size is not None:
            zero = np.zeros(new_size)
            for i in range(min(result.shape[0],new_size)):
                zero[i]=zero[i]+result[i]
            zero = zero.astype('float32')
    
    if verbose:
        print ("Predicting: {} {}".format(txt,zero))
    return zero

In [71]:
INPUT_SIZE = 7680
funnel_encoder(txt="Hellow world  ccc",new_size=INPUT_SIZE).shape[0]


Predicting: Hellow world  ccc [-0.0286085  -0.03808539  0.00674396 ...  0.          0.
  0.        ]


7680

In [73]:
from tqdm import tqdm
import numpy as np

Y = []
X = []
for (twt,lab) in tqdm(zip(df['tweet'],df['label'])):
    x = funnel_encoder(txt=twt,new_size=INPUT_SIZE,verbose=False)
    if x.shape[0] <= INPUT_SIZE:
        X.append(x)
        Y.append(lab)
    else:
        print ("warn: ",x.shape)
X = np.array(X)
Y = np.array(Y)

print (X.shape,Y.shape)

31962it [16:59, 31.34it/s]


(31962, 7680) (31962,)


In [79]:
X = np.array(X).astype('float32')
Y = np.array(Y).astype('float32')

print (X.shape,Y.shape)

(31962, 7680) (31962,)


In [80]:

class MyDataset(torch.utils.data.Dataset):
    
    def __init__(self, text_list, label_list):
        self.text_list = text_list
        self.label_list = label_list
    
    def __len__(self):
        return len(self.text_list)
    
    def __getitem__(self, index):
        return self.text_list[index],self.label_list[index]


In [81]:
import gc
from torch import nn
from torch import optim
import torch.nn.functional as F

def execute (runnable):
    if torch.cuda.device_count()>0:
        print ("  --> Let's work with cuda")
        device = torch.device("cuda:0")
        gc.collect()
        torch.cuda.empty_cache()
        if runnable is not None:
            with torch.cuda.device(0):
                return runnable(device)
        return device
    else:
        print ("  --> Let's work with CPU")
        device = torch.device("cpu")
        if runnable is not None:
            return runnable(device)
        return device

for x in range (3):
    execute(None)

def to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    print ("  --> Copied data: ",data.shape)
    return data.to(device, non_blocking=True)

def load_to_device(loader,device):
    load_list=[]
    for i, data in enumerate(loader):
        try:
            load_list.append((to_device(data[0],device), to_device(data[1],device)))
        except Exception as e:
            print (e)
    return load_list

BATCH_SIZE = 1000

train_dataset = MyDataset(text_list=X,label_list=Y)

# for (i,(x,y)) in enumerate(train_dataset):
#     if i%100==0:
#         print (x,y)

train_set_loader = torch.utils.data.DataLoader(
        train_dataset,batch_size=BATCH_SIZE,shuffle=True)

train_list = load_to_device(train_set_loader,execute(None))


  --> Let's work with cuda
  --> Let's work with cuda
  --> Let's work with cuda
  --> Let's work with cuda
  --> Copied data:  torch.Size([1000, 7680])
  --> Copied data:  torch.Size([1000])
  --> Copied data:  torch.Size([1000, 7680])
  --> Copied data:  torch.Size([1000])
  --> Copied data:  torch.Size([1000, 7680])
  --> Copied data:  torch.Size([1000])
  --> Copied data:  torch.Size([1000, 7680])
  --> Copied data:  torch.Size([1000])
  --> Copied data:  torch.Size([1000, 7680])
  --> Copied data:  torch.Size([1000])
  --> Copied data:  torch.Size([1000, 7680])
  --> Copied data:  torch.Size([1000])
  --> Copied data:  torch.Size([1000, 7680])
  --> Copied data:  torch.Size([1000])
  --> Copied data:  torch.Size([1000, 7680])
  --> Copied data:  torch.Size([1000])
  --> Copied data:  torch.Size([1000, 7680])
  --> Copied data:  torch.Size([1000])
  --> Copied data:  torch.Size([1000, 7680])
  --> Copied data:  torch.Size([1000])
  --> Copied data:  torch.Size([1000, 7680])
  --> C

In [82]:

class Perceptron(nn.Module):
    
    def __init__(self, input_size, hidden_size, output_size, hidden_layer_quantity, batch_norm_enabled=True):
        
        super().__init__()
        self.layers = []
        
        # Making the basic perceptron:---------------------------------
        dropout_base = 0.05
        self.layers.append(nn.Linear(input_size,hidden_size))
        self.layers.append(nn.Dropout( dropout_base ))
        
        for cnt in range(0,hidden_layer_quantity):
            self.layers.append(nn.Linear(hidden_size,hidden_size))
            if batch_norm_enabled == True:
                self.layers.append(nn.BatchNorm1d(hidden_size))
            drop_coef = (1.0/hidden_layer_quantity) * cnt + 2*dropout_base
            drop_coef = 0.80 if drop_coef>0.80 else drop_coef
            self.layers.append(nn.Dropout( drop_coef ))
        self.layers.append(nn.Linear(hidden_size,output_size))

        # Registering modules for torch:-------------------------------
        for cnt,layer in enumerate(self.layers):
            key = "layer{}".format(cnt)
            self.add_module(key,layer)

    def forward(self, xtr):
        for cnt,layer in enumerate(self.layers):
            if cnt<len(self.layers)-1:
                xtr = F.leaky_relu(layer(xtr))
            else :
                xtr = F.sigmoid(layer(xtr))
        return xtr

In [90]:

perceptron = Perceptron(INPUT_SIZE,300,1,3).to(execute(None))

for idx, module in enumerate(perceptron.modules()):
     print(idx, '->', module)

criteria = nn.MSELoss()
optimizer = optim.SGD(perceptron.parameters(), lr=0.005, momentum=0.001)

  --> Let's work with cuda
0 -> Perceptron(
  (layer0): Linear(in_features=7680, out_features=300, bias=True)
  (layer1): Dropout(p=0.05, inplace=False)
  (layer2): Linear(in_features=300, out_features=300, bias=True)
  (layer3): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer4): Dropout(p=0.1, inplace=False)
  (layer5): Linear(in_features=300, out_features=300, bias=True)
  (layer6): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer7): Dropout(p=0.43333333333333335, inplace=False)
  (layer8): Linear(in_features=300, out_features=300, bias=True)
  (layer9): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer10): Dropout(p=0.7666666666666666, inplace=False)
  (layer11): Linear(in_features=300, out_features=1, bias=True)
)
1 -> Linear(in_features=7680, out_features=300, bias=True)
2 -> Dropout(p=0.05, inplace=False)
3 -> Linear(in_features=300, out_features=300, bias=Tr

In [91]:

def adapt_perceptron (device):
    epoch_max_num = 100000
    history=[]
    for epoch_num in range(epoch_max_num):
        running_loss = 0.0
        for train_batch in train_list:
            x_train, y_train = train_batch[0], train_batch[1]
            #y_train[y_train==1.0]=0.75
            optimizer.zero_grad()
            y_pred = perceptron(x_train)
            loss = criteria(y_pred, y_train)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        history.append((epoch_num,running_loss))
        if epoch_num % 1000==0 :
            print("Epoch [{}/{}] loss: {}".format(epoch_num,epoch_max_num,running_loss))

    return history

train_history = execute(runnable=adapt_perceptron)


  --> Let's work with cuda
Epoch [0/100000] loss: 7.724609360098839
Epoch [1000/100000] loss: 2.160012736916542
Epoch [2000/100000] loss: 2.140989728271961
Epoch [3000/100000] loss: 2.1292636804282665
Epoch [4000/100000] loss: 2.1215590313076973
Epoch [5000/100000] loss: 2.1142961494624615
Epoch [6000/100000] loss: 2.108599830418825
Epoch [7000/100000] loss: 2.1046392619609833
Epoch [8000/100000] loss: 2.1010694429278374
Epoch [9000/100000] loss: 2.0984586887061596
Epoch [10000/100000] loss: 2.0962159894406796
Epoch [11000/100000] loss: 2.0944766625761986
Epoch [12000/100000] loss: 2.0931642167270184
Epoch [13000/100000] loss: 2.0919413790106773
Epoch [14000/100000] loss: 2.0911302231252193
Epoch [15000/100000] loss: 2.090341307222843
Epoch [16000/100000] loss: 2.0898041538894176
Epoch [17000/100000] loss: 2.0893528535962105
Epoch [18000/100000] loss: 2.088976301252842
Epoch [19000/100000] loss: 2.088675446808338
Epoch [20000/100000] loss: 2.088412892073393
Epoch [21000/100000] loss: 2